#### Import dependencies.

In [1]:
from table import Table, MapReduce

## Task 1 
Having in mind data/clicks dataset with "date" column, count how many clicks there were for each date and write the results to data/total_clicks dataset with "date" and "count" columns.

1. Let's read data into a table.

In [18]:
number_of_nodes = 1
path1 = '/home/filambr/Documents/vinted_homework/data/clicks/'
schema1 = {'date':str, 'user_id':int, 'click_target':str}
# if file that is being read does not have key specified in the schema 'Nan' will apear in the table
# if file has other keys that are not specified in the shcema, they will be omited 
table1 = Table(folder_path = path1, schema = schema1, number_of_nodes = number_of_nodes) 
table1.show()

{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile'}
{'date': '2017-12-10', 'user_id': 7, 'click_target': 'ad'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad'}


2. Let's apply count_grouped_by method.

In [19]:
daily_click = table1.count_groupe_by('date','click_target','daily_clicks')
daily_click.show(1000) # Big number is entered to show all the results since talbe isn't to big

{'date': '2017-12-20', 'daily_clicks': 13}
{'date': '2017-12-19', 'daily_clicks': 8}
{'date': '2017-12-21', 'daily_clicks': 7}
{'date': '2017-12-10', 'daily_clicks': 4}
{'date': '2017-12-12', 'daily_clicks': 12}
{'date': '2017-12-17', 'daily_clicks': 12}
{'date': '2017-12-13', 'daily_clicks': 9}
{'date': '2017-12-14', 'daily_clicks': 10}
{'date': '2017-12-16', 'daily_clicks': 10}
{'date': '2017-12-11', 'daily_clicks': 4}
{'date': '2017-12-15', 'daily_clicks': 6}
{'date': '2017-12-18', 'daily_clicks': 5}


Count_groupe_by method is implemented using MapReduce as follows:

In [20]:
mapreduce = MapReduce(number_of_nodes)
mapped = mapreduce.mapper(table1.data, key = 'date')
counted = mapreduce.counter_grouped(mapped, 'data', 'count_target', 'daily_clicks')
for row in counted:
    print(row)

{'data': '2017-12-20', 'daily_clicks': 13}
{'data': '2017-12-19', 'daily_clicks': 8}
{'data': '2017-12-21', 'daily_clicks': 7}
{'data': '2017-12-10', 'daily_clicks': 4}
{'data': '2017-12-12', 'daily_clicks': 12}
{'data': '2017-12-17', 'daily_clicks': 12}
{'data': '2017-12-13', 'daily_clicks': 9}
{'data': '2017-12-14', 'daily_clicks': 10}
{'data': '2017-12-16', 'daily_clicks': 10}
{'data': '2017-12-11', 'daily_clicks': 4}
{'data': '2017-12-15', 'daily_clicks': 6}
{'data': '2017-12-18', 'daily_clicks': 5}


#### ! Keep in mind that count_group_by counts entries, so it will also count 'None' as an entry, while agg_group_by will return an error, so None values should be removed prior to aggregation.
* Example of how to use agg_group_by is shown below

In [5]:
nonsence_table = table1.agg_groupe_by(key='date',value='user_id',alias='sum_of_id_daily', agg_type='sum') # sum, min and max agg_type are supported
nonsence_table.show(1000) # Big number is entered to show all the results since talbe isn't to big


print('__________________________')
mapreduce = MapReduce(number_of_nodes)
mapped = mapreduce.mapper(table1.data, key = 'date')
aggregated = mapreduce.aggregate_grouped(mapped, 'data', 'user_id', 'sum_of_id_daily', 'sum')
for row in aggregated:
    print(row)

{'date': '2017-12-20', 'sum_of_id_daily': 58}
{'date': '2017-12-19', 'sum_of_id_daily': 39}
{'date': '2017-12-21', 'sum_of_id_daily': 26}
{'date': '2017-12-10', 'sum_of_id_daily': 27}
{'date': '2017-12-12', 'sum_of_id_daily': 39}
{'date': '2017-12-17', 'sum_of_id_daily': 42}
{'date': '2017-12-13', 'sum_of_id_daily': 39}
{'date': '2017-12-14', 'sum_of_id_daily': 49}
{'date': '2017-12-16', 'sum_of_id_daily': 48}
{'date': '2017-12-11', 'sum_of_id_daily': 11}
{'date': '2017-12-15', 'sum_of_id_daily': 27}
{'date': '2017-12-18', 'sum_of_id_daily': 18}
__________________________
{'data': '2017-12-20', 'sum_of_id_daily': 58}
{'data': '2017-12-19', 'sum_of_id_daily': 39}
{'data': '2017-12-21', 'sum_of_id_daily': 26}
{'data': '2017-12-10', 'sum_of_id_daily': 27}
{'data': '2017-12-12', 'sum_of_id_daily': 39}
{'data': '2017-12-17', 'sum_of_id_daily': 42}
{'data': '2017-12-13', 'sum_of_id_daily': 39}
{'data': '2017-12-14', 'sum_of_id_daily': 49}
{'data': '2017-12-16', 'sum_of_id_daily': 48}
{'data'

3. Let's write final table a file in data/total_clicks.

In [21]:
path_out = 'data/total_clicks/part-001.csv'
daily_click.write_to_csv(path_out) 

## Task 2  

There are two datasets:

* data/users dataset with columns "id" and "country"
* data/clicks dataset with columns "date", "user_id" and "click_target"

We'd like to produce a new dataset called data/filtered_clicks that includes only those clicks that belong to users from Lithuania (country=LT).

1. As before lets read data into a Tables.

In [23]:
number_of_nodes = 2

path1 = '/home/filambr/Documents/vinted_homework/data/clicks/'
schema1 = {'date':str, 'user_id':int, 'click_target':str}
table1 = Table(folder_path = path1, schema = schema1,  number_of_nodes=number_of_nodes) 
print(f'Table from {path1}')
table1.show()

path2 = '/home/filambr/Documents/vinted_homework/data/users/'
schema2 = {'id': int, 'city':str,'country': str}
table2 = Table(folder_path = path2, schema = schema2, number_of_nodes=number_of_nodes) 
print(f'Table from {path2}')
table2.show()

Table from /home/filambr/Documents/vinted_homework/data/clicks/
{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile'}
{'date': '2017-12-10', 'user_id': 7, 'click_target': 'ad'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad'}
Table from /home/filambr/Documents/vinted_homework/data/users/
{'id': 4, 'city': 'Vilnius', 'country': 'LT'}
{'id': 5, 'city': 'Berlin', 'country': 'DE'}
{'id': 6, 'city': 'Munich', 'country': 'DE'}
{'id': 101, 'city': 'Hamburg', 'country': 'DE'}
{'id': 102, 'city': 'Kaunas', 'country': 'LT'}


2. Lets join tables on 'user_id' and 'id'

In [24]:
joined_table = table1.join(table2, 'user_id', 'id')
joined_table.show()

{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': 3, 'click_target': 'item', 'id': 3, 'city': 'None', 'country': 'LT'}


Join method is implemented using MapReduce like this:

In [9]:
mapreduce = MapReduce(2)
mapped_self = mapreduce.mapper(table1.data, 'user_id')
mapped_other = mapreduce.mapper(table2.data, 'id')
joined = mapreduce.join(mapped_self , mapped_other)

for i in range(5):
    print(joined[i])

{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': 3, 'click_target': 'item', 'id': 3, 'city': 'None', 'country': 'LT'}


3. Since we are only interested in clicks from Lithuanian ('LT') users lets apply a filter method to isolate 'LT' users.

In [10]:
filttered_table = joined_table.filter('country', 'LT')
filttered_table.show()

{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': 3, 'click_target': 'item', 'id': 3, 'city': 'None', 'country': 'LT'}


Method filter is implemented using MapReduce like this:

In [11]:
mapreduce = MapReduce(2)
mapped = mapreduce.mapper(joined_table.data, 'country')
filtered = mapreduce.filter('LT', mapped)
for i in range(5):
    print(filtered[i])

{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad', 'id': 3, 'city': 'None', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': 3, 'click_target': 'item', 'id': 3, 'city': 'None', 'country': 'LT'}


To avoid getting unnecessary columns in the final table use Select and/or Drop method like shown below.

In [25]:
number_of_nodes = 2
path1 = '/home/filambr/Documents/vinted_homework/data/clicks/'
schema1 = {'date':str, 'user_id':int, 'click_target':str}
table1 = Table(folder_path = path1, schema = schema1,  number_of_nodes=number_of_nodes)


path2 = '/home/filambr/Documents/vinted_homework/data/users/'
schema2 = {'id': int, 'city':str,'country': str}
table2 = Table(folder_path = path2, schema = schema2, number_of_nodes=number_of_nodes).select(['id','country'])

joined_table = table1.join(table2, 'user_id', 'id').drop(['id'])

filttered_table = joined_table.filter('country', 'LT')
filttered_table.show()



{'date': '2017-12-20', 'user_id': 3, 'click_target': 'ad', 'country': 'LT'}
{'date': '2017-12-19', 'user_id': 3, 'click_target': 'ad', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': 3, 'click_target': 'profile', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': 3, 'click_target': 'ad', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': 3, 'click_target': 'item', 'country': 'LT'}


Select and drop are implemented like this:

In [26]:
mapreduce = MapReduce(2)
selected = mapreduce.select(table2.data, ['id', 'country'])
for i in range(5):
    print(selected[i])

print('______________________________')

mapreduce = MapReduce(2)
droped = mapreduce.drop(table1.data, ['click_target'])
for i in range(5):
    print(droped[i])

{'id': 4, 'country': 'LT'}
{'id': 5, 'country': 'DE'}
{'id': 6, 'country': 'DE'}
{'id': 101, 'country': 'DE'}
{'id': 102, 'country': 'LT'}
______________________________
{'date': '2017-12-20', 'user_id': 3}
{'date': '2017-12-19', 'user_id': 3}
{'date': '2017-12-21', 'user_id': 3}
{'date': '2017-12-10', 'user_id': 7}
{'date': '2017-12-12', 'user_id': 3}


4. Finally lets filter out rows where 'country' is 'LT' and drop column 'country' since it will become redundant.


In [27]:
filttered_table.write_to_csv('data/filtered_clicks/part-001.csv') # this will work if filtered_clicks folder is not present
